# Using the pytorch example for CIFAR10

#### Note on downloading the dataset:
Downloaded the cifar10 dataset manually using curl, but the code below will download it if it missing from the datasets folder.

In [19]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

d = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [20]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./datasets', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [21]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, padding = 1)
        self.conv2 = nn.Conv2d(6, 16, 3, padding = 1)
        self.conv3 = nn.Conv2d(16, 32, 3, padding = 1)
        self.conv4 = nn.Conv2d(32, 64, 3, padding = 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.pool4 = nn.MaxPool2d(4, 4)
        self.fc1 = nn.Linear(64, 30)
        self.fc2 = nn.Linear(30, 10)

    def forward(self, x):
        x = self.pool2(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool2(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
loss_list = []
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(d), data[1].to(d)

        optimizer.zero_grad()

        outputs = net(inputs).to(d)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  
            loss_list.append(running_loss / 2000)
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.256
[1,  4000] loss: 1.989
[1,  6000] loss: 1.777
[1,  8000] loss: 1.679
[1, 10000] loss: 1.584
[1, 12000] loss: 1.526
[2,  2000] loss: 1.466
[2,  4000] loss: 1.409
[2,  6000] loss: 1.353
[2,  8000] loss: 1.356
[2, 10000] loss: 1.299
[2, 12000] loss: 1.293
[3,  2000] loss: 1.237
[3,  4000] loss: 1.210
[3,  6000] loss: 1.186
[3,  8000] loss: 1.161
[3, 10000] loss: 1.179
[3, 12000] loss: 1.167
[4,  2000] loss: 1.101
[4,  4000] loss: 1.085
[4,  6000] loss: 1.096
[4,  8000] loss: 1.068
[4, 10000] loss: 1.077
[4, 12000] loss: 1.055
[5,  2000] loss: 1.030
[5,  4000] loss: 1.012
[5,  6000] loss: 1.010
[5,  8000] loss: 1.004
[5, 10000] loss: 1.014
[5, 12000] loss: 1.020
[6,  2000] loss: 0.961
[6,  4000] loss: 0.960
[6,  6000] loss: 0.970
[6,  8000] loss: 0.958
[6, 10000] loss: 0.964
[6, 12000] loss: 0.985
[7,  2000] loss: 0.920
[7,  4000] loss: 0.920
[7,  6000] loss: 0.916
[7,  8000] loss: 0.941
[7, 10000] loss: 0.932
[7, 12000] loss: 0.958
[8,  2000] loss: 0.884
[8,  4000] 

In [24]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 65 %


# Failed experiments

#### Experiment 1

```python
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, padding = 1)
        self.conv2 = nn.Conv2d(6, 16, 3, padding = 1)
        self.conv3 = nn.Conv2d(16, 32, 3, padding = 1)
        self.conv4 = nn.Conv2d(32, 64, 3, padding = 1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.pool4 = nn.MaxPool2d(4, 4)
        self.fc1 = nn.Linear(64, 30)
        self.fc2 = nn.Linear(30, 10)

    def forward(self, x):
        x = self.pool2(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool2(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()
```
After 10 epochs got to 65% on test. 16mins on gpu

#### Test 2
